In [9]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Protocol.KDF import scrypt
from Crypto.Util.Padding import pad, unpad

def generate_symmetric_key(password, salt, key_size=16):
    key = scrypt(password, salt, key_size, N=2 ** 14, r=8, p=1)
    return key


def encrypt_message(message, key):
    iv = get_random_bytes(BLOCK_SIZE)  # Generate a random IV
    cipher = AES.new(key, AES.MODE_CBC, iv)
    ciphertext = cipher.encrypt(pad(message, BLOCK_SIZE))
    return iv + ciphertext  # Prepend IV to the ciphertext


def decrypt_message(encrypted_data, key):
    iv = encrypted_data[:BLOCK_SIZE]  # Extract IV from the encrypted data
    ciphertext = encrypted_data[BLOCK_SIZE:]  # Extract ciphertext
    cipher = AES.new(key, AES.MODE_CBC, iv)
    decrypted_message = unpad(cipher.decrypt(ciphertext), BLOCK_SIZE)
    return decrypted_message

In [ ]:
import socket

MESSAGE_PATH = r"C:\Users\adina\Desktop\תקיית_עבודות\אבטחת רשתות\messages1.txt"
BLOCK_SIZE = AES.block_size

def dummy_client(host, port):
    try:
        with open(MESSAGE_PATH, 'rb') as f:
            a = str(f.read())
            params = a.split(' ')
            password = params[3]
            salt_password = params[4]
            symmetric_key = generate_symmetric_key(password=password, salt=salt_password)
    except FileNotFoundError:
        print("File not found. Please check the path:", MESSAGE_PATH)
        return
    except IndexError:
        print("Invalid file format. Make sure the file contains the required parameters.")
        return
    except Exception as e:
        print("An error occurred while reading the file:", e)
        pass

    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as client_socket:
            client_socket.bind((host, port))
            client_socket.listen()
            print(f"Dummy client listening on {host}:{port}")

            while True:
                conn, addr = client_socket.accept()
                with conn:
                    print(f"Connected by {addr}")
                    data = conn.recv(1024)
                    if not data:
                        break
                    msg = decrypt_message(data,symmetric_key)
                    print("Received:", msg)
    except OSError as e:
        print("Failed to create socket:", e)
        pass
    except Exception as e:
        print("An error occurred during socket communication:", e)
        pass

if __name__ == "__main__":
    dummy_client("127.0.0.1", 5000)


Dummy client listening on 127.0.0.1:5000
Connected by ('127.0.0.1', 52183)
Received: b'G\xca\x97}+\x8e\xc9\x00\xb7A\x9f\x86\xca\xd2|gHello man, I use gdb'
Connected by ('127.0.0.1', 52411)
Received: b'\x00\x98\xb6\x8e\x85\x9a\x01\x85q9\xf8N:%\x98\xe0Bro\r\r\r\r\r\r\r\r\r\r\r\r\rf\x9ak{WS\xa3\xdb\x83Q\xb1\x7f\x98\xc9\xd0gWqt\xff^g\xbb\xc80$\xafi\xcf\xc4~\x0fsee ya '
Connected by ('127.0.0.1', 52463)
Received: b'\ruj\xee\x19l\xdb\xb7\xc8\xf7\x82\xa7\xeeS\xc43exit\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c&\xf9YJ\xa1UQ\xc3\x00m\x8d\xca\x8b\xf7\\\xb2\x15\x9bu\x9a\xb2\xa5z\xf3\x95\r\x04\xd2\xd6\xc1\x88\xa9exit'
